In [26]:
# Some initial data exploration, to see what the sentiment data looks like!

In [1]:
# load some prereq libraries
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('TKAgg')
from matplotlib import pyplot as plt
from sklearn import cross_validation

In [2]:
dftrain = pd.read_csv('output/Sentiment.csv')

In [29]:
dftrain

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0000,yes,1.0000,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0000,yes,1.0000,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0000,yes,1.0000,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona
5,6,Ted Cruz,0.6332,yes,1.0000,Positive,0.6332,None of the above,1.0000,NaN,...,NaN,228,NaN,NaN,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",NaN,2015-08-07 09:54:44 -0700,629697194283499520,NaN,Central Time (US & Canada)
6,7,No candidate mentioned,1.0000,yes,1.0000,Negative,0.6761,FOX News or Moderators,1.0000,NaN,...,NaN,17,NaN,NaN,RT @warriorwoman91: I liked her and was happy ...,NaN,2015-08-07 09:54:44 -0700,629697192383672320,North Georgia,Eastern Time (US & Canada)
7,8,No candidate mentioned,1.0000,yes,1.0000,Neutral,1.0000,None of the above,1.0000,NaN,...,NaN,0,NaN,NaN,Going on #MSNBC Live with @ThomasARoberts arou...,NaN,2015-08-07 09:54:44 -0700,629697192169750528,New York NY,Eastern Time (US & Canada)
8,9,Ben Carson,1.0000,yes,1.0000,Negative,0.6889,None of the above,0.6444,NaN,...,NaN,0,NaN,NaN,Deer in the headlights RT @lizzwinstead: Ben C...,NaN,2015-08-07 09:54:44 -0700,629697190219243524,NaN,Pacific Time (US & Canada)
9,10,No candidate mentioned,0.4594,yes,0.6778,Negative,0.6778,None of the above,0.4594,NaN,...,NaN,1,NaN,NaN,RT @NancyOsborne180: Last night's debate prove...,NaN,2015-08-07 09:54:42 -0700,629697185093824512,NaN,NaN


In [16]:
plt.hist(dftrain['retweet_count'], bins=[0, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
plt.show()

In [3]:
senti_ratings = zip(dftrain['sentiment'], dftrain['sentiment_confidence'])
pos_ratings = []
neg_ratings = []
neu_ratings = []
for s, sc in senti_ratings:
    if s == "Positive":
        pos_ratings.append(sc)
    elif s == "Negative":
        neg_ratings.append(sc)
    elif s == "Neutral":
        neu_ratings.append(sc)
print "Mean Neg: {0}".format(np.nanmean(neg_ratings))
print "Median Neg: {0}".format(np.nanmedian(neg_ratings))
print "Variance Neg: {0}".format(np.nanvar(neg_ratings))

print "Mean Neu: {0}".format(np.nanmean(neu_ratings))
print "Median Neu: {0}".format(np.nanmedian(neu_ratings))
print "Variance Neu: {0}".format(np.nanvar(neu_ratings))

print "Mean Pos: {0}".format(np.nanmean(pos_ratings))
print "Median Pos: {0}".format(np.nanmedian(pos_ratings))
print "Variance Pos: {0}".format(np.nanvar(pos_ratings))

Mean Neg: 0.800326892735
Median Neg: 0.6983
Variance Neg: 0.0419280284585
Mean Neu: 0.669858911521
Median Neu: 0.6629
Variance Neu: 0.043137490511
Mean Pos: 0.714484123435
Median Pos: 0.6742
Variance Pos: 0.0544544156916


In [22]:
# 2. We want to see a histogram of value counts
dftrain.tweet_location.value_counts().plot(kind='bar')
plt.title('Histogram of events by event type')
plt.show()

In [118]:
# unique user count
users = dftrain['name']
user_set = set(users)
print len(user_set)

9829


In [3]:
# lets import some tokenizers and other things to split up the data
from stop_words import get_stop_words
import collections
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
import word2vec

In [4]:
tokenizer = RegexpTokenizer(r'\w+')
p_stemmer = PorterStemmer()

en_stop = get_stop_words('en')

In [8]:
df = dftrain['text']
count = 0
for e in df:
    raw = e.decode('utf-8').lower()
    tokens = tokenizer.tokenize(raw)
    stopped_tokens = [i for i in tokens if not i in en_stop]
    texts = [p_stemmer.stem(i) for i in stopped_tokens]
    count = count + len(texts)
print count

170899


In [558]:
df = dftrain['text']
texts_all = []
for e in df:
    raw = e.decode('utf-8').lower()
    tokens = tokenizer.tokenize(raw)
    stopped_tokens = [i for i in tokens if i not in en_stop]
    texts = [p_stemmer.stem(i) for i in stopped_tokens]
    texts_all.append(texts)

In [539]:
df = dftrain['text']
texts_all = []
avg_wordcount = []
for e in df:
    raw = e.decode('utf-8').lower()
    tokens = tokenizer.tokenize(raw)
    stopped_tokens = [i for i in tokens if i not in en_stop]
    texts = [p_stemmer.stem(i) for i in stopped_tokens]
    texts_all.append(tokens)
    avg_wordcount.append(len(tokens))
np.mean(avg_wordcount)

18.010886021195301

In [250]:
count = 0
for t in texts_all:
    for w in t:
        count = count + 1
print count

249829


In [253]:
# build the vocab to train the word-2-vec on.
f = open('text-vocab', 'w')
for t in texts_all:
    for w in t:
        try:
            f.write(w)
            f.write(' ')
        except:
            continue

In [255]:
f = open('text-vocab', 'r')
text = f.read()
len(process_text_no_stem(text))

249822

In [559]:
vocabulary = [item for sublist in texts_all for item in sublist]
# print vocabulary
vocabulary_set = list(set(vocabulary))
# print vocabulary_set

In [560]:
len(vocabulary_set)

15354

In [540]:
# lets create a counter of the number of times a set of words was used
wordfreq = [vocabulary.count(w) for w in vocabulary_set]
# print "Pairs\n" + str(zip(vocabulary_set, wordfreq))

KeyboardInterrupt: 

In [26]:
from scipy import stats
stats.describe(wordfreq)

DescribeResult(nobs=15354, minmax=(1, 14424), mean=11.13058486387912, variance=24175.349128987691, skewness=66.01208466386859, kurtosis=5395.4062977117665)

In [39]:
pairs = zip(vocabulary_set, wordfreq)
vocab_sorted_by_count = sorted(pairs, key=lambda x: x[1])

In [27]:
import collections

In [541]:
x = collections.Counter(vocabulary)
l = range(len(x.keys()))
plt.bar(l, x.values(), align='center')
plt.xticks(l, x.keys())
plt.show()

In [56]:
for i in range(len(vocab_sorted_by_count)):
    w, c = vocab_sorted_by_count[i]
    if c > 11:
        print i
        print w
        break

13901
argu


In [54]:
print np.mean(wordfreq)
print len(wordfreq)

11.1305848639
15354


In [59]:
# vocab_sorted_by_count[13901:14000]
# all relevant words?

In [254]:
def process_text_no_stem(e):
    raw = e.decode('utf-8').lower()
    tokens = tokenizer.tokenize(raw)
    return tokens

In [18]:
# takes in the text, and outputs the stemmed / tokeized / stop worded tokens
def process_text(e):
    raw = e.decode('utf-8').lower()
    tokens = tokenizer.tokenize(raw)
    stopped_tokens = [i for i in tokens if not i in en_stop]
    texts = [p_stemmer.stem(i) for i in stopped_tokens]
    return texts

In [406]:
def create_sentence_vector(m, e):
    raw = e.decode('utf-8').lower()
    tokens = tokenizer.tokenize(raw)
    sentence_vector = None
    for t in tokens:
        try:
            if not sentence_vector:
                sentence_vector = m[t]
            else:
                sentence_vector = sentence_vector + m[t]
        except:
            continue
    return sentence_vector

In [19]:
def generate_plot_wordfreq(vocab):
    x = collections.Counter(vocab)
    l = range(len(x.keys()))
    plt.bar(l, x.values(), align='center')
    plt.xticks(l, x.keys())
    plt.show()

In [20]:
def generate_hist_wordfreq(wf):
    plt.hist(wf, bins=[0, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
    plt.show()

In [21]:
def create_pairs(vb_set, wf):
    pairs = zip(vb_set, wf)
    vocab_sorted_by_count = sorted(pairs, key=lambda x: x[1])
    vocab_sorted_by_count.reverse()
    return vocab_sorted_by_count

In [229]:
positive_tokens = []
neutral_tokens = []
negative_tokens = []
sentiment_text = zip(dftrain['text'], dftrain['sentiment'])
for text, sentiment in sentiment_text:
    if sentiment == 'Positive':
        positive_tokens = positive_tokens + process_text(text)
    elif sentiment == 'Negative':
        negative_tokens = negative_tokens + process_text(text)
    else:
        neutral_tokens = neutral_tokens + process_text(text)


In [23]:
def tokenize_text_sentiment(textsenti_list):
    positive_tokens = []
    neutral_tokens = []
    negative_tokens = []
    for text, sentiment in textsenti_list:
        if sentiment == 'Positive':
            positive_tokens = positive_tokens + process_text(text)
        elif sentiment == 'Negative':
            negative_tokens = negative_tokens + process_text(text)
        else:
            neutral_tokens = neutral_tokens + process_text(text)
    return positive_tokens, neutral_tokens, negative_tokens

In [395]:
def create_sentiment_sets(textsenti_list):
    positive_vectors = []
    neutral_vectors = []
    negative_vectors = []
    for text, sentiment in textsenti_list:
        if sentiment == 'Positive':
            positive_vectors.append(create_sentence_vector(model, text))
        elif sentiment == 'Negative':
            negative_vectors.append(create_sentence_vector(model, text))
        else:
            neutral_vectors.append(create_sentence_vector(model, text))
    return positive_vectors, neutral_vectors, negative_vectors

In [230]:
positive_set = set(positive_tokens)
negative_set = set(negative_tokens)
neutral_set = set(neutral_tokens)

In [231]:
print len(positive_set)
print len(negative_set)
print len(neutral_set)

4113
10572
6804


In [22]:
len(set.intersection(positive_set, negative_set))

2461

In [538]:
len(negative_set - positive_set - neutral_set)

6568

In [23]:
len(set.intersection(positive_set, neutral_set))

2131

In [24]:
len(set.intersection(negative_set, neutral_set))

3418

In [49]:
set.intersection(positive_set, negative_set)

{u'four',
 u'jihad',
 u'sleep',
 u'lolgop',
 u'hate',
 u'marcorubio',
 u'whose',
 u'voter',
 u'tweet',
 u'jamiaw',
 u'list',
 u'lord',
 u'worth',
 u'digit',
 u'kudo',
 u'everi',
 u'risk',
 u'compassion',
 u'rise',
 u'voic',
 u'govern',
 u'politician',
 u'month',
 u'school',
 u'marthamaccallum',
 u'straight',
 u'enjoy',
 u'bill',
 u'speci',
 u'direct',
 u'lrihendri',
 u'nail',
 u'second',
 u'street',
 u'empow',
 u'n',
 u'colmorrisdavi',
 u'even',
 u'hide',
 u'christ',
 u'neg',
 u'asid',
 u'hp',
 u'schism',
 u'frank',
 u'new',
 u'ever',
 u'liberti',
 u'told',
 u'intellectu',
 u'hero',
 u'men',
 u'drew',
 u'hardbal',
 u'jon',
 u'lgbt',
 u'china',
 u'chewbacca',
 u'path',
 u'100',
 u'waronwomen',
 u'disrespect',
 u'daughter',
 u'plannedparenthood',
 u'personifi',
 u'thefriddl',
 u'militari',
 u'anymor',
 u'credit',
 u'ingrahamangl',
 u'smoke',
 u'studi',
 u'dre',
 u'sorri',
 u'controversi',
 u'golden',
 u'campaign',
 u'newspap',
 u'reagan',
 u'cruz2016',
 u'brought',
 u'invas',
 u'attitud'

In [60]:
neutral_set - positive_set - negative_set

{u'fa9w5lhar',
 u'dowgehdbgl',
 u'kwdaybois',
 u'q2tt9filhh',
 u'sleet',
 u'ygmgbav',
 u'macauley',
 u'9rd7gdzagg',
 u'chrisrugab',
 u'veyi4rwlq2',
 u'shojcsdzr8',
 u'kgkyijrww9',
 u'meisterburg',
 u'evanmcmurri',
 u'2bvmwdqbki',
 u'shopaholic_918',
 u'19xpnnv2la',
 u'hedg',
 u'mrru7b0bbw',
 u'docthompsonshow',
 u'barbhayn',
 u'deli',
 u'void',
 u'allincl',
 u'gcmaf',
 u'xy3tbz78dd',
 u'ngomsiv',
 u'rajiniv',
 u'ww483zw323',
 u'slz',
 u'ywvxhtbhew',
 u'berniesanders2016',
 u'jhweissmann',
 u'showcas',
 u'phiw1jkuap',
 u'herbert',
 u'd_copperfield',
 u'oooh',
 u'armjn6vwpg',
 u'reinvent',
 u'marynicholsca',
 u'bradbannon',
 u'slipper',
 u'allbuton',
 u'greatnessny',
 u'v1ct0rcr0cc0',
 u'6pm',
 u'xrafwtxlpz',
 u'aggreg',
 u'blasphemi',
 u'shepnewsteam',
 u'illumin',
 u'isocialfanz',
 u'1dg4euqe8d',
 u'spokesman',
 u'bmorebrian',
 u'stayinform',
 u'caitlinchri',
 u'n1ur11y1gz',
 u'qtzejscxa',
 u'muwuzw9smd',
 u'clqzqebor7',
 u'jasmine_elys',
 u'cannabiscultur',
 u'niktakanuka',
 u'dorp',


In [25]:
positive_wordfreq = [positive_tokens.count(w) for w in positive_set]
negative_wordfreq = [negative_tokens.count(w) for w in negative_set]
neutral_wordfreq = [neutral_tokens.count(w) for w in neutral_set]

In [47]:
create_pairs(positive_set, positive_wordfreq)

[(u'gopdeb', 2360),
 (u'rt', 1203),
 (u't', 746),
 (u'http', 608),
 (u'co', 561),
 (u'trump', 457),
 (u's', 448),
 (u'rwsurfergirl', 308),
 (u'realdonaldtrump', 303),
 (u'debat', 262),
 (u'night', 197),
 (u'rubio', 172),
 (u'amp', 169),
 (u'last', 166),
 (u'tedcruz', 164),
 (u'look', 154),
 (u'cruz', 153),
 (u'candid', 150),
 (u'fox', 148),
 (u'need', 146),
 (u'get', 142),
 (u'news', 137),
 (u'thank', 135),
 (u'job', 126),
 (u'think', 126),
 (u'will', 119),
 (u'question', 116),
 (u'foxnew', 114),
 (u'carson', 112),
 (u'watch', 109),
 (u'great', 108),
 (u'like', 105),
 (u'megynkelli', 104),
 (u'carlyfiorina', 104),
 (u'said', 104),
 (u'rate', 102),
 (u'bush', 99),
 (u'love', 98),
 (u'best', 97),
 (u're', 96),
 (u'gop', 93),
 (u'go', 92),
 (u'm', 89),
 (u'right', 88),
 (u'ask', 87),
 (u'can', 87),
 (u'presid', 84),
 (u'kasich', 84),
 (u'won', 83),
 (u'say', 83),
 (u'one', 82),
 (u'next', 81),
 (u'set', 80),
 (u'good', 80),
 (u'know', 78),
 (u'g', 76),
 (u'expos', 76),
 (u'togeth', 75),
 

In [48]:
create_pairs(negative_set, negative_wordfreq)

[(u'gopdeb', 8942),
 (u'rt', 5023),
 (u't', 3167),
 (u'http', 1823),
 (u'co', 1640),
 (u'rwsurfergirl', 1510),
 (u's', 1385),
 (u'trump', 1124),
 (u'fox', 1069),
 (u'candid', 758),
 (u'amp', 718),
 (u'debat', 708),
 (u'news', 679),
 (u'realdonaldtrump', 636),
 (u'just', 609),
 (u'like', 567),
 (u'jeb', 519),
 (u'don', 512),
 (u'can', 509),
 (u'peopl', 499),
 (u'gop', 497),
 (u'question', 485),
 (u'megynkelli', 480),
 (u'night', 460),
 (u'last', 453),
 (u'get', 431),
 (u'foxnew', 415),
 (u'republican', 409),
 (u'bush', 407),
 (u'want', 396),
 (u'ask', 345),
 (u'think', 344),
 (u'chri', 341),
 (u'one', 339),
 (u'watch', 329),
 (u'god', 312),
 (u'm', 312),
 (u'women', 311),
 (u'donald', 308),
 (u'realli', 300),
 (u'say', 295),
 (u'wallac', 291),
 (u'presid', 287),
 (u'talk', 281),
 (u'right', 270),
 (u'tri', 263),
 (u'need', 258),
 (u'pick', 255),
 (u'go', 251),
 (u'american', 244),
 (u'time', 244),
 (u'speak', 243),
 (u'fair', 240),
 (u'doesn', 239),
 (u'will', 230),
 (u'hear', 226),
 (u

In [29]:
generate_plot_wordfreq(positive_tokens)

In [35]:
generate_hist_wordfreq(neutral_wordfreq)

In [366]:
# setup word2vec
word2vec.word2phrase('text-vocab', 'text8-phrases', verbose=True)
# word2vec.word2clusters('text8', 'text8-clusters.txt', 100, verbose=True)
word2vec.word2vec('text8-phrases', 'text8.bin', size=5000, verbose=True)

model = word2vec.load('text8.bin')

Starting training using file text-vocab
Words processed: 200K     Vocab size: 100K  
Vocab size (unigrams + bigrams): 58835
Words in train file: 249822
Starting training using file text8-phrases
Vocab size: 3626
Words in train file: 208876
Alpha: 0.001207  Progress: 99.09%  Words/thread/sec: 17.37k  

In [ ]:
# We are going to code in the generative model
# algorithm for predicting sentiment.


# The idea behind this model is that we assume that the words of the sentence came from some distribution
# Idea is you will have three context distributions. 
# A neutral context distribution, a positive context distribution, and a negative context distribution
# Lets make the assumption that sentences are formed by drawing words from a poisson distribution
# Let the probability of any classification be the probability of drawing the words from
# each of the three distributions, with the max probability being the classification prediction.
# Every word is present in one of the three contexts. If a word is not present in that context, its probability is
# defined to be the average of the distribution, so as not to prevent words from shifting contexts, but
# some penalty be realized for the words lack of presence.

# each word will be represented by an unique number to allow for identification of the word.
# we are really trying to model the wf of the word from each distribution more so than the word from
# each distribution.

In [367]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(dftrain['text'], dftrain['sentiment'],
                                                                    test_size=0.4, random_state=0)

In [368]:
from scipy.stats import alpha as sc_alpha
from scipy.stats import norm as nm
from scipy.optimize import curve_fit
from scipy.misc import factorial
from numpy import linalg as LA
from sklearn.metrics import zero_one_loss
from sklearn.preprocessing import normalize
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import confusion_matrix

In [29]:
def poisson(k, lambd):
    return(lambd**k/factorial(k))*np.exp(-lambd)

In [85]:
# collections.Counter(positive_wordfreq)
X_test

2134     RT @NoChristie16: #ChrisChristie blasts sanctu...
1654     Even the media doesn't get it,  No one won las...
9138     RT @RWSurferGirl: We the American people pick ...
10561    RT @jamiaw: "The purpose of the military is to...
2919     #GOPDebate Maybe they were right to try &amp; ...
654      RT @RepublicanPunk: GOP has 0 candidates who d...
4872     RT @mikepolkjr: "I'm trying to answer your acc...
4842     RT @MichaelLucchese: Welcome to the #Thunderdo...
1751     I honestly really want to hear what Ben Carson...
11494    RT @SupermanHotMale: Hey Chris Wallace, These ...
11540    This is like the TV show "Review" right?  They...
201      RT @realDonaldTrump: "@FrankLuntz: I'm getting...
8268     Using Trump to troll liberals may be my new fa...
12252    RT @RWSurferGirl: The candidates don't have to...
10517    Scott Walker: I'm certainly an imperfect man *...
8151     RT @1catherinesiena: @glennbeck @PatandStu @Fo...
4004     #Out Thank u for thoughtful reflection on #GOP.

In [242]:
from sklearn.decomposition import TruncatedSVD
from mpl_toolkits.mplot3d import Axes3D

In [369]:
m_neg = []
for v in negative_set:
    try:
        vec = model[v]
        m_neg.append(vec)
    except:
        continue

m_pos = []
for v in positive_set:
    try:
        vec = model[v]
        m_pos.append(vec)
    except:
        continue  

m_neu = []
for v in neutral_set:
    try:
        vec = model[v]
        m_neu.append(vec)
    except:
        continue
    
m_neg = np.array(m_neg)
m_neu = np.array(m_neu)
m_pos = np.array(m_pos)

print m_neg.shape
print m_neu.shape
print m_pos.shape

(1561, 5000)
(1408, 5000)
(1203, 5000)


In [370]:
svd = TruncatedSVD(n_components=3, random_state=42)
m_neg_new = svd.fit_transform(m_neg)
m_neu_new = svd.fit_transform(m_neu)
m_pos_new = svd.fit_transform(m_pos)

print m_neg_new.shape
print m_neu_new.shape
print m_pos_new.shape

(1561, 3)
(1408, 3)
(1203, 3)


In [371]:
m_neg.shape

(1561, 5000)

In [529]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

xs = m_neg_new[:, 0]
ys = m_neg_new[:, 1]
zs = m_neg_new[:, 2]
# for v in negative_set:
#     try:
#         vec = model[v][6:9]
#         xs.append(vec[0])
#         ys.append(vec[1])
#         zs.append(vec[2])
#     except:
#         continue

ax.scatter(xs, ys, zs)

xs_p = m_pos_new[:, 0]
ys_p = m_pos_new[:, 1]
zs_p = m_pos_new[:, 2]

ax.scatter(xs_p, ys_p, zs_p, c='r')

xs_ne = m_neu_new[:, 0]
ys_ne = m_neu_new[:, 1]
zs_ne = m_neu_new[:, 2]

ax.scatter(xs_ne, ys_ne, zs_ne, c='g')

plt.show()

In [131]:
def poisson_training_data(vb):
    vectors = []
    for w in vb:
        try:
            v = model[w]
            vectors.append(v)
        except:
            continue
    return vectors

In [373]:
def gaus_training_data(vb):
    vectors = []
    for w in vb:
        try:
            vec = model[w]
            vectors.append(vec)
        except:
            continue
    
    svd = TruncatedSVD(n_components=1000, random_state=42)
    vectors = svd.fit_transform(vectors)
    return vectors, svd

In [476]:
def gaus_training_data_sentence_vectors(vs):
    svd = TruncatedSVD(n_components=100, random_state=42)
    vectors = svd.fit_transform(vs)
    return vectors, svd

In [374]:
svd.transform(model['chris'].reshape(1, -1))

array([[ 0.99700513,  0.04868942,  0.03036666]])

In [375]:
def create_poisson(vb):
    vbs = poisson_training_data(vb)
    alph, loc, scale = sc_alpha.fit(vbs)
    alpha = sc_alpha(alph, loc=loc, scale=scale)
    return alpha

In [376]:
def create_gaus(vb):
    vbs, svd = gaus_training_data(vb)
    loc, scale = nm.fit(vbs)
    gaus = nm(loc=loc, scale=scale)
    return gaus, svd

In [410]:
def create_gaus_sentence(vb):
    vbs, svd = gaus_training_data_sentence_vectors(vb)
    loc, scale = nm.fit(vbs)
    gaus = nm(loc=loc, scale=scale)
    return gaus, svd

In [377]:
class PoissonWordClassifier:
    
#     self.pos_poi, self.neu_poi, self.neg_poi = None, None, None
#     self.pos_set, self.pos_wf, self.neu_set = None, None, None
#     self.neu_wf, self.neg_set, self.neg_wf = None, None, None
    
    def __init__(self, x_train, y_train, x_test, y_test):
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
        self.train()
    
    def train(self):
        pos, neu, neg = tokenize_text_sentiment(zip(self.x_train, self.y_train))
        self.pos_set, self.neu_set, self.neg_set = set(pos), set(neu), set(neg)
#         self.pos_wf = [positive_tokens.count(w) for w in self.pos_set]
#         self.neg_wf = [negative_tokens.count(w) for w in self.neg_set]
#         self.neu_wf = [neutral_tokens.count(w) for w in self.neu_set]
        
        self.pos_poi, self.pos_svd = create_gaus(self.pos_set)
        self.neu_poi, self.neu_svd = create_gaus(self.neu_set)
        self.neg_poi, self.neg_svd = create_gaus(self.neg_set)
            
    # score takes in a single word "x" and identifies the probability that it came from one of the three buckets
    def score(self, x):
        # find x in each of the three sets, get its word_count in that set, and use that to get the probability
        try:      
            w_vec = model[x]
    #             print w_vec
            w_vec_pos = self.pos_svd.transform(w_vec.reshape(1, -1))
            w_vec_neg = self.neg_svd.transform(w_vec.reshape(1, -1))
            w_vec_neu = self.neu_svd.transform(w_vec.reshape(1, -1))
            pos_sc = self.pos_poi.pdf(w_vec_pos)
            neu_sc = self.neu_poi.pdf(w_vec_neu)
            neg_sc = self.neg_poi.pdf(w_vec_neg)
            return pos_sc, neu_sc, neg_sc
        except:
            reshaped_ones = np.ones(10,).reshape(10,)
            v_norm = reshaped_ones / 10.0
            return v_norm, v_norm, v_norm

    def classify(self, sentence):
        tokens = process_text(sentence)
        s_pos = 1.0
        s_neu = 1.0
        s_neg = 1.0
        for t in tokens:
            (pos_sc, neu_sc, neg_sc) = self.score(t)
            pos_sc = LA.norm(pos_sc, 1)
            neu_sc = LA.norm(neu_sc, 1)
            neg_sc = LA.norm(neg_sc, 1)
            s_pos = s_pos * pos_sc
            s_neu = s_neu * neu_sc
            s_neg = s_neg * neg_sc
        
        if s_pos > s_neu and s_pos > s_neg:
            return 0
        if s_neu > s_pos and s_neu > s_neg:
            return 1
        if s_neg > s_pos and s_neg > s_neu:
            return 2
    
    def predict(self, texts):
        y_predict = []
        for text in texts:
            y_predict.append(self.classify(text))
        return y_predict
    
    def test(self):
        y_predict = self.predict(self.x_test)
        y_true = []
        for l in y_test:
            if l == 'Positive':
                y_true.append(0)
            if l == 'Neutral':
                y_true.append(1)
            if l == 'Negative':
                y_true.append(2)
        
        error = zero_one_loss(y_true, y_predict)
        print error
        cm = confusion_matrix(y_true, y_predict)
        plot_confusion_matrix(cm)
    

In [378]:
pwc = PoissonWordClassifier(X_train, y_train, X_test, y_test)

In [379]:
pwc.score("trump")[2]

array([[  4.82512215e-114,   4.25068626e-004,   1.03317356e-012,
          1.95594996e-023,   6.09871472e-044,   6.71379149e+000,
          1.09062046e-012,   1.15849442e+001,   3.43107321e-001,
          2.91670214e-001,   1.25290554e+001,   1.26247036e+001,
          1.25728181e+001,   1.23691819e+001,   1.24222312e+001,
          1.26219391e+001,   1.25388359e+001,   1.21744434e+001,
          1.25440648e+001,   1.24926934e+001,   1.24312810e+001,
          1.24457738e+001,   1.26246052e+001,   1.25898509e+001,
          1.24475923e+001,   1.26233947e+001,   1.26127834e+001,
          1.25730305e+001,   1.26057573e+001,   1.26074305e+001,
          1.26241232e+001,   1.26246828e+001,   1.26138543e+001,
          1.26204448e+001,   1.26231168e+001,   1.26169283e+001,
          1.26246671e+001,   1.26192556e+001,   1.26131833e+001,
          1.26077711e+001,   1.26144030e+001,   1.26170706e+001,
          1.26230583e+001,   1.26131738e+001,   1.26211048e+001,
          1.26226795e+001

In [380]:
pwc.test()

0.583168138403


In [365]:
plt.show()

In [200]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [201]:
labels = ['Positive', 'Neutral', 'Negative']

In [217]:
from sklearn.cross_validation import StratifiedKFold

In [218]:
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([0, 0, 1, 1])
skf = StratifiedKFold(y, n_folds=2)
for train_index, test_index in skf:
    print train_index

[1 3]
[0 2]


In [477]:
# Redo the gaussian model, but with sentences that have their vectors aggregated.
class SentenceVectorClassifier:
    
#     self.pos_poi, self.neu_poi, self.neg_poi = None, None, None
#     self.pos_set, self.pos_wf, self.neu_set = None, None, None
#     self.neu_wf, self.neg_set, self.neg_wf = None, None, None
    
    def __init__(self, x_train, y_train, x_test, y_test):
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
            
    def train(self):
        pos, neu, neg = create_sentiment_sets(zip(self.x_train, self.y_train))
#         self.pos_set, self.neu_set, self.neg_set = set(pos), set(neu), set(neg)
#         self.pos_wf = [positive_tokens.count(w) for w in self.pos_set]
#         self.neg_wf = [negative_tokens.count(w) for w in self.neg_set]
#         self.neu_wf = [neutral_tokens.count(w) for w in self.neu_set]
        
        self.pos_poi, self.pos_svd = create_gaus_sentence(pos)
        self.neu_poi, self.neu_svd = create_gaus_sentence(neu)
        self.neg_poi, self.neg_svd = create_gaus_sentence(neg)
            
    # score takes in a single word "x" and identifies the probability that it came from one of the three buckets
    def score(self, svector):
        # find x in each of the three sets, get its word_count in that set, and use that to get the probability
        try:      
            w_vec_pos = self.pos_svd.transform(svector.reshape(1, -1))
            w_vec_neg = self.neg_svd.transform(svector.reshape(1, -1))
            w_vec_neu = self.neu_svd.transform(svector.reshape(1, -1))
            pos_sc = self.pos_poi.pdf(w_vec_pos)
            neu_sc = self.neu_poi.pdf(w_vec_neu)
            neg_sc = self.neg_poi.pdf(w_vec_neg)
            return pos_sc, neu_sc, neg_sc
        except:
            reshaped_ones = np.ones(10,).reshape(10,)
            v_norm = reshaped_ones / 10.0
            return v_norm, v_norm, v_norm

    def classify(self, sentence):
        svector = create_sentence_vector(model, sentence)
        s_pos = 1.0
        s_neu = 1.0
        s_neg = 1.0
        (pos_sc, neu_sc, neg_sc) = self.score(svector)
        s_pos = LA.norm(pos_sc)
        s_neu = LA.norm(neu_sc)
        s_neg = LA.norm(neg_sc)
                
        if s_pos > s_neu and s_pos > s_neg:
            return 0
        if s_neu > s_pos and s_neu > s_neg:
            return 1
        if s_neg > s_pos and s_neg > s_neu:
            return 2
    
    def predict(self, texts):
        y_predict = []
        for text in texts:
            y_predict.append(self.classify(text))
        return y_predict
    
    def test(self):
        y_predict = self.predict(self.x_test)
        y_true = []
        for l in y_test:
            if l == 'Positive':
                y_true.append(0)
            if l == 'Neutral':
                y_true.append(1)
            if l == 'Negative':
                y_true.append(2)
        
        error = zero_one_loss(y_true, y_predict)
        print error
        cm = confusion_matrix(y_true, y_predict)
        plot_confusion_matrix(cm)
    

In [478]:
svc = SentenceVectorClassifier(X_train, y_train, X_test, y_test)
svc.train()

In [479]:
svc.test()

0.770409082718


In [473]:
print len(y_test)

5549


In [480]:
plt.show()

In [486]:
set(list(dftrain['subject_matter']))

{nan,
 'Abortion',
 'FOX News or Moderators',
 'Foreign Policy',
 'Gun Control',
 'Healthcare (including Medicare)',
 'Immigration',
 'Jobs and Economy',
 'LGBT issues',
 'None of the above',
 'Racial issues',
 'Religion',
 "Women's Issues (not abortion though)"}

In [491]:
# lets try understanding the part of speech distributions
import nltk
texts = zip(dftrain['text'], dftrain['sentiment'])
# sentiment = dftrain['sentiment']
positive_tags = []
negative_tags = []
neutral_tags = []
for text, s in texts:
    tags = nltk.pos_tag(text)
    if s == 'Positive':
        positive_tags = positive_tags + [t for (word, t) in tags]
    if s == 'Neutral':
        neutral_tags = neutral_tags + [t for (word, t) in tags]
    if s == 'Negative':
        negative_tags = negative_tags + [t for (word, t) in tags]

In [501]:
cp = collections.Counter(positive_tags)
cn = collections.Counter(negative_tags)
cnn = collections.Counter(neutral_tags)

In [502]:
N = len(cp)
values = cp.values()
ind = np.arange(N)
width = 0.35

fig, ax = plt.subplots()
rects1 = ax.bar(ind, values, width, color='r')

# add some text for labels, title and axes ticks
ax.set_ylabel('Counts')
ax.set_title('Counts by POS_TAG')
ax.set_xticks(ind + width)
ax.set_xticklabels(cp.keys())


def autolabel(rects):
    # attach some text labels
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                '%d' % int(height),
                ha='center', va='bottom')

autolabel(rects1)

plt.show()

In [503]:
N = len(cn)
values = cn.values()
ind = np.arange(N)
width = 0.35

fig, ax = plt.subplots()
rects1 = ax.bar(ind, values, width, color='r')

# add some text for labels, title and axes ticks
ax.set_ylabel('Counts')
ax.set_title('Counts by POS_TAG')
ax.set_xticks(ind + width)
ax.set_xticklabels(cn.keys())


def autolabel(rects):
    # attach some text labels
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.05*height,
                '%d' % int(height),
                ha='center', va='bottom')

autolabel(rects1)

plt.show()

In [519]:
# lets try understanding the part of speech distributions
import nltk
texts = zip(dftrain['tweet_location'], dftrain['sentiment'])
# sentiment = dftrain['sentiment']
p_cand = []
n_cand = []
neu_cand = []
for c, s in texts:
    if s == 'Positive':
        p_cand.append(c)
    if s == 'Neutral':
        neu_cand.append(c)
    if s == 'Negative':
        n_cand.append(c)

In [525]:
set(n_cand) - set(neu_cand) - set(p_cand)

set()

In [524]:
texts = zip(dftrain['subject_matter'], dftrain['sentiment'])
# sentiment = dftrain['sentiment']
p_cand = []
n_cand = []
neu_cand = []
for c, s in texts:
    if s == 'Positive':
        p_cand.append(c)
    if s == 'Neutral':
        neu_cand.append(c)
    if s == 'Negative':
        n_cand.append(c)

In [528]:
set(n_cand)

{nan,
 'Abortion',
 'FOX News or Moderators',
 'Foreign Policy',
 'Gun Control',
 'Healthcare (including Medicare)',
 'Immigration',
 'Jobs and Economy',
 'LGBT issues',
 'None of the above',
 'Racial issues',
 'Religion',
 "Women's Issues (not abortion though)"}

In [530]:
## K-nearest neighbors Collab filtering here we come!
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import zero_one_loss
from sklearn.neighbors import KNeighborsClassifier

In [531]:
def create_feature_vector(tokens, candidate, location, subject):
    # returns the feature vector that represents this
    v = np.zeros(len(vocabulary_set) + 3)
    for t in tokens:
        v[vocabulary_set.index(t)] = 1
    # get candidate id
    v[len(vocabulary_set)] = candidate_set.index(candidate)
    v[len(vocabulary_set) + 1] = location_set.index(location)
    v[len(vocabulary_set) + 2] = subject_set.index(subject)
    return v

In [533]:
def label_to_num(ys):
    y_true = []
    for l in ys:
        if l == 'Positive':
            y_true.append(0)
        if l == 'Neutral':
            y_true.append(1)
        if l == 'Negative':
            y_true.append(2)
    return y_true

In [536]:
k = 10
skf = StratifiedKFold(label_to_num(dftrain['sentiment']),n_folds=k)
averageError = 0.0
for train_index, test_index in skf:
    dftrain_cols = zip(dftrain['text'], dftrain['candidate'], dftrain['tweet_location'], dftrain['subject_matter'])
    X_train, X_test = dftrain['text'][train_index], dftrain['text'][test_index]
    y_train, y_test = dftrain['sentiment'][train_index], dftrain['sentiment'][test_index]
    
    # create the LDA transformation first
    m = np.array([np.zeros(len(vocabulary_set) + 3) for i in range(len(X_train))])
    for i in range(len(X_train)):
        tokens = process_text(X_train.iloc[i])
        candidate = dftrain['candidate'][train_index[i]]
        tweet_location = dftrain['tweet_location'][train_index[i]]
        sm = dftrain['subject_matter'][train_index[i]]
        m[i] = create_feature_vector(tokens, candidate, tweet_location, sm)
    
    m_test = np.array([np.zeros(len(vocabulary_set) + 3) for i in range(len(X_test))])
    for i in range(len(X_test)):
        tokens = process_text(X_test.iloc[i])
        candidate = dftrain['candidate'][test_index[i]]
        tweet_location = dftrain['tweet_location'][test_index[i]]
        sm = dftrain['subject_matter'][test_index[i]]
        m_test[i] = create_feature_vector(tokens, candidate, tweet_location, sm)
    print m_test
    
    neigh = KNeighborsClassifier(n_neighbors=15)
    neigh.fit(m, y_train)
    
    y_pred = neigh.predict(m_test)
    error = zero_one_loss(label_to_num(y_test), label_to_num(y_pred))
    print error
    averageError += (1./k) * error
print "Average error: %4.2f%s" % (100 * averageError,'%')

ValueError: u'kinghilar' is not in list

In [542]:
import random

In [544]:
def classify():
    num = random.random()
    if num < 0.33:
        return "Positive"
    elif 0.34 < num < 0.66:
        return "Neutral"
    else:
        return "Negative"

In [545]:
# POS_TAG RF classifier
k = 3
skf = StratifiedKFold(label_to_num(dftrain['sentiment']),n_folds=k)
averageError = 0.0
for train_index, test_index in skf:
    dftrain_cols = zip(dftrain['text'], dftrain['candidate'], dftrain['tweet_location'], dftrain['subject_matter'])
    X_train, X_test = dftrain['text'][train_index], dftrain['text'][test_index]
    y_train, y_test = dftrain['sentiment'][train_index], dftrain['sentiment'][test_index]
    
    y_pred = []
    for x in X_test:
        y_pred.append(classify())
    error = zero_one_loss(label_to_num(y_test), label_to_num(y_pred))
    print error
    averageError += (1./k) * error
print "Average error: %4.2f%s" % (100 * averageError,'%')

0.679783783784
0.663638330089
0.668829764222
Average error: 67.08%


In [546]:
import nltk
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [547]:
def create_feature_vector_pos(tokens):
    v = np.zeros(len(pos_set))
    tags = nltk.pos_tag(tokens)
    for w, t in tags:
        v[pos_sl.index(t)] = v[pos_sl.index(t)] + 1
    
    return v

In [549]:
candidate_list = []
for c in dftrain['candidate']:
    candidate_list.append(c)
candidate_set = list(set(candidate_list))

subject_list = []
for s in dftrain['subject_matter']:
    subject_list.append(s)
subject_set = list(set(subject_list))

location_list = []
for l in dftrain['tweet_location']:
    location_list.append(l)
location_set = list(set(location_list))

In [551]:
pos_set = set()
for s in dftrain['text']:
#     print s
    raw = s.decode('utf-8').lower()
    tokens = tokenizer.tokenize(raw)
    tags = nltk.pos_tag(tokens)
#     print tags
    for w, t in tags:
        pos_set.add(t)
print pos_set

set(['PRP$', 'VBG', 'VBD', 'VBN', 'VBP', 'WDT', 'JJ', 'WP', 'VBZ', 'DT', 'RP', '$', 'NN', 'FW', 'TO', 'PRP', 'RB', 'NNS', 'NNP', 'VB', 'WRB', 'CC', 'PDT', 'RBS', 'RBR', 'CD', 'EX', 'IN', 'WP$', 'MD', 'NNPS', 'JJS', 'JJR', 'SYM', 'UH'])


In [553]:
pos_sl = list(pos_set)

In [556]:
def create_token_vector(tokens):
    v = np.zeros(len(vocabulary_set))
    for t in tokens:
        v[vocabulary_set.index(t)] = 1
    return v

In [563]:
def create_candidate_vector(candidate):
    v = np.zeros(len(candidate_set))
    v[candidate_set.index(candidate)] = 1
    return v

In [565]:
def create_subject_vector(sm):
    v = np.zeros(len(subject_set))
    v[subject_set.index(sm)] = 1
    return v

In [568]:
def create_location_vector(l):
    v = np.zeros(len(location_set))
    v[location_set.index(l)] = 1
    return v

In [570]:
# Feature selection time
# Feature selection time
# POS_TAG RF classifier
k = 10
skf = StratifiedKFold(label_to_num(dftrain['sentiment']),n_folds=k)
averageError = 0.0
for train_index, test_index in skf:
    dftrain_cols = zip(dftrain['text'], dftrain['candidate'], dftrain['tweet_location'], dftrain['subject_matter'])
    X_train, X_test = dftrain['text'][train_index], dftrain['text'][test_index]
    y_train, y_test = dftrain['sentiment'][train_index], dftrain['sentiment'][test_index]
    
    # create the LDA transformation first
    z_array = np.zeros(len(location_set))
    m = np.array([z_array for i in range(len(X_train))])
    for i in range(len(X_train)):
#         tokens = process_text_no_stem(X_train.iloc[i])
        tokens_stemmed = process_text(X_train.iloc[i])
        candidate = dftrain['candidate'][train_index[i]]
        tweet_location = dftrain['tweet_location'][train_index[i]]
        sm = dftrain['subject_matter'][train_index[i]]
#         v1 = create_feature_vector_expanded(tokens_stemmed, candidate, tweet_location, sm)
#         m[i] = np.concatenate((create_feature_vector_pos(tokens), v1))
        m[i] = create_location_vector(tweet_location)
    print m
#     model = LDA(n_topics=13, random_state=0)
#     model.fit(m.astype(int))

    F, p_val = f_classif(m, y_train)
#     f = open("f-scores.txt", "w")
#     f.write("Length of POS: {0}, Length of Vocab: {1}, Length of Candidate: {2}, Length of location: {3}".format(
#             len(pos_set), len(vocabulary_set), len(candidate_set), len(location_set)))
#     for i in range(len(F)):
#         f.write("Score: {0}, Column: {1}\n".format(F[i], i))
#     f.close()
    print np.nanmean(F)
    print np.nanmedian(F)
    print np.nanvar(F)
    break
    
#     clf = RandomForestClassifier(random_state=0)
#     clf.fit(m, y_train)
#     y_pred = clf.predict(m_test)
#     error = zero_one_loss(label_to_num(y_test), label_to_num(y_pred))
#     print error
#     averageError += (1./k) * error
# print "Average error: %4.2f%s" % (100 * averageError,'%')
print "done"

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
1.08325474013
0.512103385692
1.21735740523
done


/Users/abkhanna/Documents/workspace/Princeton/cos424/a2/lib/python2.7/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [  27   56   62  105  107  120  131  151  155  169  181  184  195  201  204
  222  223  238  240  253  254  262  263  272  279  284  292  310  312  327
  368  380  386  402  403  409  410  416  419  452  457  472  490  497  501
  507  509  518  531  533  535  539  550  552  579  586  587  616  643  648
  679  681  682  685  697  699  703  707  714  720  723  724  728  741  756
  771  792  810  815  823  851  862  884  896  900  912  914  920  935  938
  941  967  983  986 1017 1019 1027 1045 1073 1089 1104 1112 1115 1117 1125
 1127 1130 1134 1150 1153 1173 1267 1268 1277 1311 1326 1345 1347 1360 1368
 1376 1382 1429 1457 1462 1488 1516 1525 1533 1538 1543 1554 1560 1577 1581
 1586 1592 1595 1629 1633 1639 1653 1654 1667 1670 1672 1673 1678 1702 1726
 1729 1742 1746 1764 1770 1786 1821 1827 1831 1837 1843 1854 1891 1896 1900
 19